#### Reference
https://colab.research.google.com/drive/1-Cg7L4yzCspmRjzJk_vP8G1m4SHNRwB4?usp=sharing

#### Candidate Models
- meta-llama/Llama-3.2-3B
- google/flan-t5-small

#### Set Up

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments
from peft import prepare_model_for_kbit_training, LoraConfig
from trl import SFTTrainer
import torch

from utils import load_dataset

In [2]:
import os
from dotenv import load_dotenv
from huggingface_hub import login

load_dotenv()
login(token=os.getenv('hugging_face_token'))

In [3]:
dataset = load_dataset('../data/data_01_processed.csv')

#### Finetuning

In [4]:
model_name = "meta-llama/Llama-3.2-3B"

compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=compute_dtype,
            bnb_4bit_use_double_quant=True,)
model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config, device_map={"": 0})
model = prepare_model_for_kbit_training(model)

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True, add_eos_token=True)
tokenizer.pad_token = tokenizer.unk_token
tokenizer.padding_side = "left"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.05,
    r=8,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules= ["down_proj","up_proj","gate_proj"])

In [ ]:
training_arguments = TrainingArguments(
        output_dir="../data/results/",
        eval_strategy="steps",
        optim="paged_adamw_8bit",
        save_steps=250,
        log_level="debug",
        logging_steps=250,
        learning_rate=1e-4,
        eval_steps=250,
        fp16=True,
        do_eval=True,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        #gradient_accumulation_steps=1,
        warmup_steps=100,
        max_steps=500,
        lr_scheduler_type="linear"
)

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset['train'],
    eval_dataset=dataset['validation'],
    peft_config=peft_config,
    processing_class=tokenizer,
    args=training_arguments
)
trainer.train()

TypeError: SFTTrainer.__init__() got an unexpected keyword argument 'max_seq_length'